In [17]:
# !pip install iapws

In [18]:
from iapws import IAPWS97 #1

In [19]:
T1 = 100+273.15

In [20]:
a = IAPWS97(T=T1, x=0.5)

In [21]:
P1 = a.P

In [22]:
h = a.h

In [23]:
s1 = a.s

In [24]:
print(f'pressure {P1}, enthalpy {h}, entropy {s1}')

pressure 0.10141797792131013, enthalpy 1547.3355921092682, entropy 4.330545689399871


In [25]:
water=IAPWS97(T=100+273.15,x=0.5)
water.P, water.h, water.s

(0.10141797792131013, 1547.3355921092682, 4.330545689399871)

In [26]:
water=IAPWS97(T=170+273.15,x=0.5)
water.Liquid.cp, water.Vapor.cp, water.Liquid.w, water.Vapor.w

(4.369498593233083, 2.5985140134188485, 1418.2631646730363, 498.7751280146938)

A closed rigid vessel contains 40% liquid water and 60% water vapor by volume. The liquid-vapor mixture is in equilibrium at 150&deg;C. What is the quality of the mixture?

$$mv = m_L v_L + m_v V_v$$

The amount of vapor relative to liquid is defined by a parameter called the *quality* of the system. The quality is the mass fraction of vapor in the system.

# Ideal Rankine Cycle
Determine the efficiency of a Rankine cycle using steam as the working fluid in which the condenser pressure is 10kPa. The boiler pressure is 2MPa. The steam leaves the boiler as saturated vapor.

![ideal rankine cycle](Images/ideal_rankine_cycle.png)

In [27]:
p1 = 0.01 # MPa
p2 = 2 # MPa
# state at point 3 is saturated vapor

In [28]:
# !pip install pyromat
# import pyromat as pm
# run below to see how pyromat is configured
# pm.config

In [29]:
# mp_water = pm.get("mp.H2O")

Look at the pump first, section 1 to 2

In [30]:
p1

0.01

In [31]:
# saturated liquid, thus x = 0
sat_liquid=IAPWS97(P=p1, x=0) #saturated liquid with known T
sat_liquid.v

w_p = sat_liquid.v*(p2-p1)*1000
print(f"Work required by pump: {round(float(w_p),1)} kJ/kg")

Work required by pump: 2.0 kJ/kg


In [32]:
h1 = sat_liquid.h
h2 = h1 + w_p
print(f'h2 = {round(h2,1)} kJ/kg')

h2 = 193.8 kJ/kg


Analyze the boiler, section 2 to 3

In [33]:
# saturated vapor, x = 1
sat_vapor=IAPWS97(P=p2, x=1)

p3 = p2
h3 = sat_vapor.h
s3 = sat_vapor.s

q_H = h3-h2

print(f"Heat input by boilder: {round(float(q_H),1)} kJ/kg")

Heat input by boilder: 2604.6 kJ/kg


Analyze the turbine, section 3 to 4

In [34]:
p4 = p1
s4 = s3
sat_vapor4=IAPWS97(P=p4,s=s4)
h4 = sat_vapor4.h

w_t = h3-h4
print(f"Work generated by turbine: {round(float(w_t),1)} kJ/kg")

Work generated by turbine: 791.7 kJ/kg


Analysis of the condenser.

In [35]:
q_L = h4 - h1

print(f"Heat rejected by the condenser: {round(float(q_L),1)} kJ/kg")

Heat rejected by the condenser: 1814.8 kJ/kg


Determine the thermal efficiency

In [36]:
eta_th = (w_t-w_p)/q_H*100

print(f"Thermal efficiency is: {round(float(eta_th),1)}%")

Thermal efficiency is: 30.3%


# Rankine with Reheat

Consider a reheat cycle utilizing steam. Steam leaves the boiler and enters the turbine at 4 MPa, 400&deg;C. After expansion in the turbine to 400kPa, the steam is reheated to 400&deg;C and then expanded in the low pressure turbine to 10kPa. Determine the cycle efficiency.  
  
![rankine with reheat](Images/rankine_with_reheat.png)

In [37]:
p3 = 4 # MPa
p2 = p3
T3 = 400 + 273.15 # kelvins
p4 = 0.4 # 400 kPa
p5 = p4
T5 = 400 + 273.15 # kelvins
p6 = 0.01 # 10 kPa
p1 = p6

Start with the pump

In [38]:
# saturated liquid from the condenser
sat_liquid1 = IAPWS97(P=p1,x=0)
v12 = sat_liquid1.v

w_p = v12 * (p2 - p1) * 1000
print(f'Pump work is {round(float(w_p),1)} kJ/kg')

Pump work is 4.0 kJ/kg


In [39]:
h2 = h1 + w_p
print(f'Enthalpy after the pump is {round(float(h2),1)} kJ/kg')

Enthalpy after the pump is 195.8 kJ/kg


Analyzing the high pressure turbine

In [40]:
sat_vap3 = IAPWS97(T=T3,P=p3)
h3 = sat_vap3.h
s3 = sat_vap3.s
s4 = s3
sat_vap4 = IAPWS97(P=p4,s=s4)
h4 = sat_vap4.h

w_hpt = h3 - h4
print(f'The work output by the high pressure turbine is {round(float(w_hpt),1)} kJ/kg')

The work output by the high pressure turbine is 528.1 kJ/kg


Analyzing the low pressure turbine

In [41]:
p5 = p4
sat_vap5 = IAPWS97(T=T5, P=p5)
h5 = sat_vap5.h
s5 = sat_vap5.s
s6 = s5
sat_vap6 = IAPWS97(P=p6, s=s6)
h6 = sat_vap6.h

w_lpt = h5 - h6
print(f"The work output by the low pressure turbine is {round(float(w_lpt),1)} kJ/kg")
print(f"Total work output by the turbine is {round(float(w_hpt+w_lpt),1)} kJ/kg")

The work output by the low pressure turbine is 769.3 kJ/kg
Total work output by the turbine is 1297.4 kJ/kg


Analyzing the boiler

In [42]:
q_H = (h3 - h2) + (h5 - h4)
print(f'The heat input by the boiler is {round(float(q_H),1)} kJ/kg')

The heat input by the boiler is 3606.1 kJ/kg


Analyzing the condenser

In [43]:
q_L = h6 - h1
print(f'The heat loss by the condenser is {round(float(q_L),1)} kJ/kg')

The heat loss by the condenser is 2312.7 kJ/kg


Calculate the thermal efficiency

In [45]:
net_work_turbine = w_hpt + w_lpt
eta_th = (net_work_turbine - w_p) / q_H * 100
print(f'The thermal efficiency is {round(float(eta_th),1)}%')

The thermal efficiency is 35.9%


# Real Rankine Cycle

A steam power plant operates on a cycle with pressures and temperatures as shown in the picture. The efficiency of the turbine is 86%, and the efficiency of the pump is 80%. Determine the thermal efficiency of this cycle.
![real rankine cycle](Images/real_rankine_cycle.png)

In [46]:
from iapws import IAPWS97 #1
import pyromat as pm

p1 = 0.01 # 10 kPa
T1 = 42+273.15
p2 = 5 # MPa
p3 = 4.8 # MPa
T3 = 40+273.15
p4 = 4 # MPa
T4 = 400+273.15
p5 = 3.8 # MPa
T5 = 380+273.15

eta_turbine = 0.86
eta_pump = 0.8

Analyze the pump

In [47]:
# Saturated liquid
sat_liquid1 = IAPWS97(P=p1,x=0)
h1 = sat_liquid1.h
v1 = sat_liquid1.v

w_pump = v1*(p2-p1)/eta_pump * 1000
h2 = h1 + w_pump
print(f"Work of the pump is {round(float(w_pump),1)} kJ/kg")
print(f"Enthalpy after the pump is {round(float(h2),1)} kJ/kg")

Work of the pump is 6.3 kJ/kg
Enthalpy after the pump is 198.1 kJ/kg


Analyze the boiler

In [48]:
# sat_liquid3 = IAPWS97(P=p3,T=T3)
sat_liquid3 = IAPWS97(T=T3,x=0)
h3 = sat_liquid3.h
vapor4 = IAPWS97(P=p4,T=T4)
h4 = vapor4.h

q_H = h4 - h3
print(f"The heat from the boiler is {round(float(q_H),1)} kJ/kg")

The heat from the boiler is 3046.8 kJ/kg


Analyze the turbine

In [49]:
p6 = p1
p6

0.01

In [62]:
vapor5 = IAPWS97(P=p5,T=T5)
h5 = vapor5.h
s5 = vapor5.s
s6s = s5

# !pip install pyXSteam
from pyXSteam.XSteam import XSteam
steamTable = XSteam(XSteam.UNIT_SYSTEM_BARE)
# print(steamTable.hL_p(5))

sL6 = steamTable.sL_p(p6)
sV6 = steamTable.sV_p(p6)
sLV6 = sV6 - sL6

x6 = (s6s - sL6) / sLV6

vapor6 = IAPWS97(P=p6,x=x6)
h6s = vapor6.h

w_turbine = eta_turbine*(h5-h6s)
print(f"The quality of the steam coming out of the turbine is {round(float(x6),4)}")
print(f"The work extracted by the turbine is {round(float(w_turbine),1)} kJ/kg")

The quality of the steam coming out of the turbine is 0.8105
The work extracted by the turbine is 894.5 kJ/kg


In [64]:
eta_th = (w_turbine - w_pump) / q_H * 100
print(f"Thermal Efficiency of the cycle is {round(float(eta_th),1)}%")

Thermal Efficiency of the cycle is 29.2%
